In [207]:
import numpy as np
import pandas as pd 
import numpy.random as rn

from scipy import optimize       # to compare

In [192]:
def annealing(random_start,
              cost_function,
              random_neighbour,
              acceptance,
              temperature,
              maxsteps=1000,
              debug=True):
    """ Optimize the black-box function 'cost_function' with the simulated annealing algorithm."""
    state = random_start()
    cost = cost_function(state)
    states, costs = [state], [cost]
    for step in range(maxsteps):
        fraction = step / float(maxsteps)
        T = temperature(fraction)
        new_state = random_neighbour()  # fraction
        new_cost = cost_function(new_state)
#         if debug: print("Step #{:>2}/{:>2} : T = {:>4.3g}, state = {:>4.3g}, cost = {:>4.3g}, new_state = {:>4.3g}, new_cost = {:>4.3g} ...".format(step, maxsteps, T, state, cost, new_state, new_cost))
        if debug and step % 100 == 0: print("Step #{:>2}/{:>2} : T = {:>4.3g} cost = {:>4.3g} new_cost = {:>4.3g} ...".format(step, maxsteps, T, cost, new_cost))          
        if acceptance_probability(cost, new_cost, T) > rn.random():
            state, cost = new_state, new_cost
            states.append(state)
            costs.append(cost)
            # print("  ==> Accept it!")
        # else:
        #    print("  ==> Reject it...")
    return state, cost_function(state), states, costs

In [182]:
def get_random_vac(max_fly):
    return np.random.randint(36, max_fly)

def random_matrix():
    aval_hours=get_D(df)
    arr = np.zeros((df.shape[0], 12), dtype=int)
    while aval_hours >= 0:
        rand_i = np.random.randint(0, arr.shape[0])
        rand_j = np.random.randint(0, arr.shape[1])
        max_fly = df['Max Fly'].iloc[rand_i]
        _vac = get_random_vac(max_fly)
        if aval_hours > _vac:
            aval_hours -= _vac
            arr[rand_i, rand_j] = _vac
        else:
            _vac = aval_hours
            arr[rand_i, rand_j] = _vac
            break
    return arr
        
# def random_step_matrix(df):
#     _df = df[[f'{i}_vac' for i in range(1, 13)]].copy()
#     return _df.applymap(lambda x: get_random_int(_max=5)).values

In [183]:
vacs = random_matrix()

In [184]:
vacs

array([[ 0,  0, 41, ...,  0,  0,  0],
       [64,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0, 41,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0, 36, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [185]:
import pickle 

with open('possible_vacation.pickle', 'rb') as f:
    possible_vacation = pickle.load(f)

In [186]:
def get_D_d(vacs):
    return vacs.sum(axis=0)

def get_D(df):
    return possible_vacation.loc['Q9', :].sum()

def get_S(df):
    return df[[f'RP{i}_score' for i in range(1, 13)]].values.sum().sum()
    
def get_S_d(vacs, df):
    a = df[[f'RP{i}_score' for i in range(1, 13)]].values
    b = vacs.astype(bool).astype(int)
    return np.multiply(a, b).sum().sum()
    
def cost_function(vacs):
    return f(vacs)

In [187]:
def cost_function(vacs):
    cost = 1 - get_S_d(vacs, df) / get_S(df)
    return cost

In [188]:
def acceptance_probability(cost, new_cost, temperature):
    if new_cost < cost:
        # print("    - Acceptance probabilty = 1 as new_cost = {} < cost = {}...".format(new_cost, cost))
        return 1
    else:
        p = np.exp(- (new_cost - cost) / temperature)
        # print("    - Acceptance probabilty = {:.3g}...".format(p))
        return p

In [189]:
def temperature(fraction):
    """ Example of temperature dicreasing as the process goes on."""
    return max(0.01, min(1, 1 - fraction))

In [193]:
state, c, states, costs = annealing(
    random_matrix,
    cost_function,
    random_matrix,
    acceptance_probability,
    temperature,
    maxsteps=100000,
    debug=True)

Step # 0/100000 : T =    1 cost = 0.695 new_cost = 0.812 ...
Step #100/100000 : T = 0.999 cost = 0.832 new_cost = 0.819 ...
Step #200/100000 : T = 0.998 cost = 0.789 new_cost = 0.785 ...
Step #300/100000 : T = 0.997 cost = 0.77 new_cost = 0.834 ...
Step #400/100000 : T = 0.996 cost = 0.76 new_cost = 0.79 ...
Step #500/100000 : T = 0.995 cost = 0.753 new_cost = 0.783 ...
Step #600/100000 : T = 0.994 cost = 0.735 new_cost = 0.717 ...
Step #700/100000 : T = 0.993 cost = 0.769 new_cost = 0.748 ...
Step #800/100000 : T = 0.992 cost = 0.806 new_cost = 0.876 ...
Step #900/100000 : T = 0.991 cost = 0.805 new_cost = 0.911 ...
Step #1000/100000 : T = 0.99 cost = 0.776 new_cost = 0.757 ...
Step #1100/100000 : T = 0.989 cost = 0.823 new_cost = 0.819 ...
Step #1200/100000 : T = 0.988 cost = 0.717 new_cost = 0.843 ...
Step #1300/100000 : T = 0.987 cost = 0.77 new_cost = 0.767 ...
Step #1400/100000 : T = 0.986 cost = 0.806 new_cost = 0.754 ...
Step #1500/100000 : T = 0.985 cost = 0.724 new_cost = 0.8

Step #12900/100000 : T = 0.871 cost = 0.74 new_cost = 0.732 ...
Step #13000/100000 : T = 0.87 cost = 0.813 new_cost = 0.814 ...
Step #13100/100000 : T = 0.869 cost = 0.677 new_cost = 0.847 ...
Step #13200/100000 : T = 0.868 cost = 0.799 new_cost = 0.789 ...
Step #13300/100000 : T = 0.867 cost = 0.765 new_cost = 0.794 ...
Step #13400/100000 : T = 0.866 cost = 0.821 new_cost = 0.807 ...
Step #13500/100000 : T = 0.865 cost = 0.692 new_cost = 0.844 ...
Step #13600/100000 : T = 0.864 cost = 0.781 new_cost = 0.874 ...
Step #13700/100000 : T = 0.863 cost = 0.767 new_cost = 0.808 ...
Step #13800/100000 : T = 0.862 cost = 0.775 new_cost = 0.728 ...
Step #13900/100000 : T = 0.861 cost = 0.767 new_cost = 0.718 ...
Step #14000/100000 : T = 0.86 cost = 0.826 new_cost = 0.769 ...
Step #14100/100000 : T = 0.859 cost = 0.772 new_cost = 0.742 ...
Step #14200/100000 : T = 0.858 cost = 0.864 new_cost = 0.716 ...
Step #14300/100000 : T = 0.857 cost = 0.779 new_cost = 0.868 ...
Step #14400/100000 : T = 0.8

Step #25600/100000 : T = 0.744 cost = 0.806 new_cost = 0.774 ...
Step #25700/100000 : T = 0.743 cost = 0.798 new_cost = 0.805 ...
Step #25800/100000 : T = 0.742 cost = 0.84 new_cost = 0.749 ...
Step #25900/100000 : T = 0.741 cost = 0.774 new_cost = 0.801 ...
Step #26000/100000 : T = 0.74 cost = 0.856 new_cost = 0.761 ...
Step #26100/100000 : T = 0.739 cost = 0.856 new_cost = 0.764 ...
Step #26200/100000 : T = 0.738 cost = 0.803 new_cost = 0.806 ...
Step #26300/100000 : T = 0.737 cost = 0.817 new_cost = 0.783 ...
Step #26400/100000 : T = 0.736 cost = 0.817 new_cost = 0.858 ...
Step #26500/100000 : T = 0.735 cost = 0.869 new_cost = 0.821 ...
Step #26600/100000 : T = 0.734 cost = 0.785 new_cost = 0.779 ...
Step #26700/100000 : T = 0.733 cost = 0.811 new_cost = 0.704 ...
Step #26800/100000 : T = 0.732 cost = 0.787 new_cost = 0.693 ...
Step #26900/100000 : T = 0.731 cost = 0.724 new_cost = 0.734 ...
Step #27000/100000 : T = 0.73 cost = 0.832 new_cost = 0.827 ...
Step #27100/100000 : T = 0.7

Step #38300/100000 : T = 0.617 cost = 0.813 new_cost = 0.806 ...
Step #38400/100000 : T = 0.616 cost = 0.799 new_cost = 0.836 ...
Step #38500/100000 : T = 0.615 cost = 0.773 new_cost =  0.8 ...
Step #38600/100000 : T = 0.614 cost = 0.783 new_cost = 0.726 ...
Step #38700/100000 : T = 0.613 cost = 0.814 new_cost = 0.825 ...
Step #38800/100000 : T = 0.612 cost = 0.707 new_cost = 0.862 ...
Step #38900/100000 : T = 0.611 cost = 0.771 new_cost = 0.841 ...
Step #39000/100000 : T = 0.61 cost = 0.78 new_cost = 0.812 ...
Step #39100/100000 : T = 0.609 cost = 0.857 new_cost = 0.693 ...
Step #39200/100000 : T = 0.608 cost = 0.818 new_cost = 0.792 ...
Step #39300/100000 : T = 0.607 cost = 0.781 new_cost = 0.792 ...
Step #39400/100000 : T = 0.606 cost = 0.861 new_cost = 0.793 ...
Step #39500/100000 : T = 0.605 cost = 0.757 new_cost = 0.781 ...
Step #39600/100000 : T = 0.604 cost = 0.756 new_cost = 0.832 ...
Step #39700/100000 : T = 0.603 cost = 0.729 new_cost = 0.73 ...
Step #39800/100000 : T = 0.60

Step #51000/100000 : T = 0.49 cost = 0.784 new_cost = 0.803 ...
Step #51100/100000 : T = 0.489 cost = 0.871 new_cost = 0.844 ...
Step #51200/100000 : T = 0.488 cost = 0.762 new_cost = 0.767 ...
Step #51300/100000 : T = 0.487 cost = 0.81 new_cost = 0.855 ...
Step #51400/100000 : T = 0.486 cost = 0.844 new_cost = 0.838 ...
Step #51500/100000 : T = 0.485 cost = 0.851 new_cost = 0.737 ...
Step #51600/100000 : T = 0.484 cost = 0.837 new_cost = 0.825 ...
Step #51700/100000 : T = 0.483 cost = 0.777 new_cost = 0.807 ...
Step #51800/100000 : T = 0.482 cost = 0.894 new_cost = 0.844 ...
Step #51900/100000 : T = 0.481 cost = 0.801 new_cost = 0.761 ...
Step #52000/100000 : T = 0.48 cost = 0.731 new_cost = 0.714 ...
Step #52100/100000 : T = 0.479 cost = 0.844 new_cost = 0.775 ...
Step #52200/100000 : T = 0.478 cost = 0.789 new_cost = 0.688 ...
Step #52300/100000 : T = 0.477 cost = 0.771 new_cost = 0.839 ...
Step #52400/100000 : T = 0.476 cost = 0.804 new_cost = 0.832 ...
Step #52500/100000 : T = 0.4

Step #63700/100000 : T = 0.363 cost = 0.789 new_cost = 0.83 ...
Step #63800/100000 : T = 0.362 cost = 0.813 new_cost = 0.765 ...
Step #63900/100000 : T = 0.361 cost = 0.893 new_cost = 0.807 ...
Step #64000/100000 : T = 0.36 cost = 0.718 new_cost = 0.81 ...
Step #64100/100000 : T = 0.359 cost = 0.719 new_cost = 0.743 ...
Step #64200/100000 : T = 0.358 cost = 0.759 new_cost = 0.768 ...
Step #64300/100000 : T = 0.357 cost = 0.731 new_cost = 0.792 ...
Step #64400/100000 : T = 0.356 cost = 0.772 new_cost = 0.801 ...
Step #64500/100000 : T = 0.355 cost = 0.807 new_cost = 0.773 ...
Step #64600/100000 : T = 0.354 cost = 0.792 new_cost = 0.819 ...
Step #64700/100000 : T = 0.353 cost = 0.781 new_cost = 0.75 ...
Step #64800/100000 : T = 0.352 cost = 0.787 new_cost = 0.835 ...
Step #64900/100000 : T = 0.351 cost = 0.778 new_cost = 0.794 ...
Step #65000/100000 : T = 0.35 cost = 0.759 new_cost = 0.775 ...
Step #65100/100000 : T = 0.349 cost = 0.777 new_cost = 0.756 ...
Step #65200/100000 : T = 0.348

Step #76400/100000 : T = 0.236 cost = 0.723 new_cost = 0.782 ...
Step #76500/100000 : T = 0.235 cost = 0.756 new_cost = 0.751 ...
Step #76600/100000 : T = 0.234 cost = 0.766 new_cost = 0.772 ...
Step #76700/100000 : T = 0.233 cost = 0.704 new_cost = 0.862 ...
Step #76800/100000 : T = 0.232 cost = 0.746 new_cost = 0.835 ...
Step #76900/100000 : T = 0.231 cost = 0.767 new_cost = 0.755 ...
Step #77000/100000 : T = 0.23 cost = 0.731 new_cost = 0.813 ...
Step #77100/100000 : T = 0.229 cost = 0.765 new_cost = 0.711 ...
Step #77200/100000 : T = 0.228 cost = 0.74 new_cost = 0.773 ...
Step #77300/100000 : T = 0.227 cost = 0.853 new_cost = 0.747 ...
Step #77400/100000 : T = 0.226 cost = 0.74 new_cost = 0.811 ...
Step #77500/100000 : T = 0.225 cost = 0.751 new_cost = 0.83 ...
Step #77600/100000 : T = 0.224 cost = 0.732 new_cost = 0.718 ...
Step #77700/100000 : T = 0.223 cost = 0.799 new_cost = 0.828 ...
Step #77800/100000 : T = 0.222 cost = 0.711 new_cost = 0.675 ...
Step #77900/100000 : T = 0.22

Step #89100/100000 : T = 0.109 cost = 0.759 new_cost = 0.787 ...
Step #89200/100000 : T = 0.108 cost = 0.799 new_cost = 0.776 ...
Step #89300/100000 : T = 0.107 cost = 0.716 new_cost = 0.856 ...
Step #89400/100000 : T = 0.106 cost = 0.73 new_cost = 0.841 ...
Step #89500/100000 : T = 0.105 cost = 0.733 new_cost = 0.855 ...
Step #89600/100000 : T = 0.104 cost = 0.716 new_cost = 0.739 ...
Step #89700/100000 : T = 0.103 cost = 0.85 new_cost = 0.847 ...
Step #89800/100000 : T = 0.102 cost = 0.779 new_cost = 0.809 ...
Step #89900/100000 : T = 0.101 cost = 0.756 new_cost = 0.843 ...
Step #90000/100000 : T =  0.1 cost = 0.738 new_cost = 0.794 ...
Step #90100/100000 : T = 0.099 cost = 0.784 new_cost = 0.777 ...
Step #90200/100000 : T = 0.098 cost = 0.816 new_cost = 0.798 ...
Step #90300/100000 : T = 0.097 cost = 0.677 new_cost = 0.772 ...
Step #90400/100000 : T = 0.096 cost = 0.723 new_cost = 0.762 ...
Step #90500/100000 : T = 0.095 cost = 0.79 new_cost = 0.744 ...
Step #90600/100000 : T = 0.09

In [196]:
state.shape

(90, 12)

In [197]:
df

,Num,Pers Level,Max Fly,Month Start,Month Max,Q1,Q2,Q3,Q4,Q5,...,RP3_score,RP4_score,RP5_score,RP6_score,RP7_score,RP8_score,RP9_score,RP10_score,RP11_score,RP12_score
140,3643,55,90,2,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
155,3798,1,90,2,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
164,4778,1,80,1,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
168,4799,70,90,1,4,0,0,0,0,0,...,141,0,0,283,0,0,0,141,0,0
175,5001,1,80,3,6,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1852,97211,1,80,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1993,99090,1,90,3,6,0,0,0,0,0,...,0,0,72,0,214,0,72,0,0,72
2009,99383,1,80,1,12,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2190,101421,1,90,3,6,0,0,0,0,0,...,0,72,0,214,0,0,72,0,0,0


In [203]:
l = []

for vac, person, person_score in zip(state, df['Num'].values, df[[f'RP{i}_score' for i in range(1, 13)]].values):
    for i, (vac_size, score) in enumerate(zip(vac, person_score), 1):
        if vac_size != 0:
            l.append([person, i, vac_size, 1 if score else 0])

In [209]:
__df = pd.DataFrame(data=l, columns=['id', 'month', 'rest_size', 'aplication'])

In [211]:
__df.to_csv('../data/res_v.0.01.csv')